# Градиентный бустинг
## Бэггинг
- В бэггинге строится несколько немного разных моделей и они усредняются

- Смещение $a_N(x)$ такое же, как у $b_n(x)$ (сложность модели не увеличится -> в бэггинге в качестве базовых моделей нужно брать сложные модели, например, глубокие решающие деревья)

- Разброс $a_N(x)$:
$$\frac{1}{N}(разброс b_n(x)) + ковариация(b_n(x), b_m(x)) $$

- Если базовые модели независимы, то разброс уменьшается в N раз (независимость = их ошибки независимы. По тому, что первая модель ошиблась на первом объекте, нельзя сказать, что вторая модель тоже ошибется на нем)

- Чем более похожи выходы базовых моделей, тем меньше эффект от построения композиции.


__Следовательно, базовые модели должны быть сложными и как можно более независимыми (непохожими).__

### Проблемы:
- Если базовая модель окажется смещенной, то и композиция не справится задачей (мало данных, не можем построить дерево, которое использует все признаки)
- Базовые модели долго обучать и применять, дорого хранить (в каждой вершине нужно перебрать все доступные предикаты, дерево глубокое - вершин много)

## Бустинг
- Простые базовые модели (деревья глубины 1-4)
- Построение композиции происходит последовательно и жадно
- Каждая следующая модель будет строиться так, чтобы максимально корректировать ошибки построенных моделей
- n+1 модель построим так, чтобы её добавление в композицию как можно сильнее уменьшило ошибку


Базовая модель:
$$a_N(x)=\sum _{n=1} ^N b_n(x)$$
Как в бэггинге, но без усреднения

Обучение n-ой модели:
$$\frac{1}{l} \sum_{i=1} ^l L(y_i, b_1(x_i)) -> min_{b_1(x)}$$
Минимизируем функционал ошибки

$$\frac{1}{l} \sum_{i=1} ^l L(y_i, a_{N-1}(x_i)+b_N(x_i)) -> min_{b_N(x)}$$

Считаем функцию потерь от правильного ответа и суммы ансамбля предыдущих n-1 моделей и новой модели $b_n$


Жадное построение - значит фиксируем все предыдущие модели, никак их не меняем

## Бустинг для MSE
$$\frac{1}{l} \sum_{i=1} ^l (a_{N-1}(x_i)+b_N(x_i)-y_i)^2 -> min_{b_N(x)}$$

Перегруппируем слагаемые
$$\frac{1}{l} \sum_{i=1} ^l (b_N(x_i) - (y_i - a_{N-1}(x_i)))^2 -> min_{b_N(x)}$$

$y_i - a_{N-1}(x_i)$ - фиксирована, так как алгоритм - жадный


Задача выглядит как обучение новой модели на среднеквадратическую ошибку с новыми значениями целевых переменных. Обозначим как 
$$s_i ^{(N)} = y_i - a_{N-1}(x_i)$$.
Это сдвиги или остатки

$$\frac{1}{l} \sum_{i=1} ^l (b_N(x_i) - s_i ^{(N)})^2 -> min_{b_N(x)}$$


В случае с MSE можно просто подменить целевую переменную


Строим первое дерево (подгоняем под исходные целевые переменные)
$$\frac{1}{l} \sum_{i=1} ^l (b_1(x_i) - y_i)^2 -> min_{b_1(x)}$$
Второе дерево:
$$\frac{1}{l} \sum_{i=1} ^l (b_2(x_i) - (y_i-b_1(x_i)))^2 -> min_{b_2(x)}$$
Третья модель:
$$\frac{1}{l} \sum_{i=1} ^l (b_3(x_i) - (y_i-b_1(x_i)-b_2(x_i)))^2 -> min_{b_3(x)}$$

### Визуализация
регрессия


<img src='images/ensembles11.png'>

В качестве базового алгоритма берём decision stumps (решающие пни). Это деревья глубины 1, в которых есть только один предикат. В зависимости от его значения выдается один их двух ответов. 

<img src='images/ensembles12.png'>

Слева визуализация самой модели, справа - остатки,
то есть отклонения уже построенной композиции от истинных целевых переменных. 
1. видно, что остатки стали чуть лучше, чем исходная выборка. Например, верхние два облака точек выровнялись друг с другом. 
2. остатки становятся еще лучше, модель тоже становится сложнее и уже учитывает правое облако точек, выдавая на нем правильные ответы. 
<img src='images/ensembles13.png'>
После 20 итерации остатки почти нулевые

В отличие от случайного леса, где было усреднение независимых моделей, бустинг легко переобучается. 


<img src='images/ensembles14.png'>


x - количество деревьев

- В случае с MSE обучение базовых моделей сводится к обычной процедуре обучения с заменой целевой переменной
- Бустинг может переобучаться, поэтому надо следить за ошибкой на тестовой выборке

### Произвольная функция потерь для бустинга
$$\frac{1}{l} \sum_{i=1} ^l L(y_i, a_{N-1}(x_i)+b_N(x_i)) -> min_{b_N(x)}$$
Если попробовать снова обучаться на остатки, как в MSE:
$$\frac{1}{l} \sum_{i=1} ^l L(y_i-a_{N-1}(x_i),b_N(x_i)) -> min_{b_N(x)}$$
Это плохая идея.
Пример: логистическая функция потерь
$$a_N(x)=sign \sum_{n=1} ^N b_n(x)$$

$$L(y,z)=log(1+exp(-yz))$$
$y$-правильный ответ, $z$- прогноз модели, $yz$ имеет смысл отступа
$$z=b_N(x_i)$$
$$y=y_i-a_{N-1}(x_i)$$
Если обучаться на остатки:
$$\frac{1}{l} \sum_{i=1} ^l log(1+exp(-(y_i-a_{N-1}(x_i)b_N(x_i)) -> min_{b_N(x)}$$

То не учитываются правильные ответы. Отсупы для корректировки неправильных ответов равноценны. 

Пример: MSLE
- аргумент логарифма может стать отрицательным
- объекты не равноценные




- Нельзя заменить обучение добавки к композиции на обучение базовой модели на отклонение от ответов
- Не учитываются особенности функции потерь

## Градиентный бустинг в общем виде
Для произвольной дифференцируемой функции потерь
$$\frac{1}{l} \sum_{i=1} ^l L(y_i, a_{N-1}(x_i)+b_N(x_i)) -> min_{b_N(x)}$$

Как посчитать, куда и как сильно сдвигать $a_{N-1}(x_i)$, чтобы уменьшить ошибку?

Можно посчитать производную:

$$s_i^{(N)}=-\frac{\partial}{\partial z}L(y_i, z)|z=a_{N-1}(x_i)$$

Первый аргумент фиксирован, считаем по второму аргументу (по прогнозу). Берем значение этой производной в точке $z=a_{N-1}(x_i)$

- Посчитаем значение производной в той точке, которая равна текущему прогнозу композиции в бустинге, и возьмем с минусом.

- Знак производной будет показывать, в какую сторону нужно сдвигать уже построенную композицию, чтобы ошибка этой композиции уменьшилась.
- Если эта производная будет отрицательной — значит, нужно уменьшать прогноз.
- Если производная будет положительной (антипроизводная) — значит, нужно увеличивать прогноз.

-Величина этой производной (ее абсолютное значение) будет показывать, насколько сильно мы уменьшим ошибку, если сдвинемся в этом направлении. Если производная будет большой по модулю, это будет означать, что, если мы даже немножко сдвинемся в эту сторону, ошибка уменьшится сильно. Если производная будет равна примерно нулю, это будет означать, что, даже если мы сдвинемся в ту сторону, ошибка, скорее всего, практически не поменяется.
- То есть если антипроизводные (производные с минусом) будут близки к нулю, то можно на этом объекте ничего особо не менять, все равно ошибка уже не изменится.


### Градиентный бустинг
Обучение N-й модели:
$$\frac{1}{l} \sum_{i=1} ^l (b_N(x_i)-s_i^{(N)})^2 -> min_{b_N(x)}$$

Сдвиги:
$$s_i^{(N)}=-\frac{\partial}{\partial z}L(y_i, z)|z=a_{N-1}(x_i)$$

Будем N-ю базовую модель $b_n(x)$ так, чтобы она минимизировала среднеквадратичную ошибку между прогнозами этой модели и сдвигами.

- Как бы градиентный спуск в пространстве ответов на обучающей выборке.

- мы знаем прогнозы нашей модели на всех объектах обучающей выборки, считаем производные по этим самым прогнозам, и эти производные в совокупности говорят, как нам сдвинуть прогнозы композиции с помощью новой модели, чтобы как можно сильнее уменьшить ошибку этой композиции. 

- Получается, что базовая модель будет делать корректировки в основном на тех объектах, где есть шанс сильно уменьшить ошибку за счет сдвига, а если сдвиги для каких-то объектов находятся около нуля, то мы и новую модель будем просить выдавать прогноз около нуля, то есть ничего не менять, потому что ошибку все равно уже не уменьшить.

- Поскольку это производные по функции потерь (производные функции потерь по аргументам), то получается, что эта задача, обучение N-й модели, учитывает особенности исходной функции потерь. 

#### Градиентный бустинг для MSE:
<img src='images/ensembles15.png'>
Получилась та же формула, что и выше


#### Градиентный бустинг для логистической функции потерь:
<img src='images/ensembles16.png'>
- По формуле видно, что если отступ большой и положительный (уверены в правильном ответе), то дробь близка к нулю
- Если же отступ большой и отрицательный (уверены в неправильном ответе), то дробь близка к +1 или -1


В итоге:
- Чтобы учесть особенности функции потерь, можно посчитать её производные в точке текущего прогноза композиции
- Базовую модель будем обучать на эти производные (со знаком минус)

### Гиперпараметры
- глубина базовых деревьев
- число деревьев N

### Проблемы бустинга
- сдвиги показывают направление, в котором надо сдвинуть композицию на всех объектах обучающей выборки
- базовые модели, как правило, очень простые
- могут не справиться с приближением этого направления

Чтобы решить эти проблемы, можно добавлять деревья в композицию с небольшим весом (шаг)

### Рандомизация по признакам
- Можно обучать деревья на случайных подмножествах признаков
- Бустинг уменьшает смещение, поэтому итоговая композиция всё равно получится качественной
- Может снизить переобучение

### Регуляризация деревьев
- введение длины шага и семплирование признаков
- штрафы за число листьев в дереве
- штрафы за величину прогнозов в листьях дерева

_Базового алгоритма градиентного бустинга недостаточно для получения сильных результатов, поэтому было разработано большое число улучшений и имплементаций. Можно улучшать тип деревьев, способ построения деревьев, подходы к регуляризации, вносить модификации в процедуру обучения деревьев_

## Имплементации градиентного бустинга 
- XGBoost
- LightGBM (leaf-wise growth, поиск порогов на основе производных)
- CatBoost (ODT)